In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
import utils;
# reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
model_path = 'data/imdb/models/'
%mkdir -p $model_path

## Setup data

We're going to look at the IMDB dataset, which contains movie reviews from IMDB, along with their sentiment. Keras comes with some helpers for this dataset.

In [4]:
from keras.datasets import imdb
idx = imdb.get_word_index()

This is the word list:

In [5]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

...and this is the mapping from id to word

In [6]:
idx2word = {v: k for k, v in idx.iteritems()}

We download the reviews using code copied from keras.datasets:

In [7]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [8]:
len(x_train)

25000

Here's the 1st review. As you see, the words have been replaced by ids. The ids can be looked up in idx2word.

In [9]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

The first word of the first review is 23022. Let's see what that is.

In [10]:
idx2word[23022]

'bromwell'

Here's the whole review, mapped from ids to words.

In [11]:
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

The labels are 1 for positive, 0 for negative.

In [12]:
labels_train[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Reduce vocab size by setting rare words to max index.

In [13]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

In [24]:
trn.shape

(25000, 500)

Look at distribution of lengths of sentences.

In [14]:
lens = np.array(map(len, trn))
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

Pad (with zero) or truncate each sentence to make consistent length.

In [15]:
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

This results in nice rectangular matrices that can be passed to ML algorithms. Reviews shorter than 500 words are pre-padded with zeros, those greater are truncated.

In [16]:
trn.shape

(25000, 500)

## Create simple models

### Single hidden layer NN

The simplest model that tends to give reasonable results is a single hidden layer net. So let's try that. Note that we can't expect to get any useful results by feeding word ids directly into a neural net - so instead we use an embedding to replace them with a vector of 32 (initially random) floats for each word in the vocab.

In [16]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [17]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 500, 32)       160000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 16000)         0           embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 100)           1600100     flatten_1[0][0]                  
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 100)           0           dense_1[0][0]                    
___________________________________________________________________________________________

In [18]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 4s - loss: 0.4826 - acc: 0.7344 - val_loss: 0.3113 - val_acc: 0.8691
Epoch 2/2
25000/25000 [==============================] - 4s - loss: 0.1996 - acc: 0.9235 - val_loss: 0.3021 - val_acc: 0.8738


### Single conv layer with max pooling

A CNN is likely to work better, since it's designed to take advantage of ordered data. We'll need to use a 1D CNN, since a sequence of words is 1D.

In [37]:
conv1 = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, dropout=0.2),
    Dropout(0.2),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [45]:
conv1.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [278]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=4, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 4s - loss: 0.4984 - acc: 0.7250 - val_loss: 0.2922 - val_acc: 0.8816
Epoch 2/4
25000/25000 [==============================] - 4s - loss: 0.2971 - acc: 0.8836 - val_loss: 0.2681 - val_acc: 0.8911
Epoch 3/4
25000/25000 [==============================] - 4s - loss: 0.2568 - acc: 0.8983 - val_loss: 0.2551 - val_acc: 0.8947
Epoch 4/4
25000/25000 [==============================] - 4s - loss: 0.2427 - acc: 0.9029 - val_loss: 0.2558 - val_acc: 0.8947


That's well past the Stanford paper's accuracy - another win for CNNs!

In [281]:
conv1.save_weights(model_path + 'conv1.h5')

In [46]:
conv1.load_weights(model_path + 'conv1.h5')

## Pre-trained vectors

You may want to look at wordvectors.ipynb before moving on.

In this section, we replicate the previous CNN, but using pre-trained embeddings.

In [26]:
def get_glove_dataset(dataset):
    """Download the requested glove dataset from files.fast.ai
    and return a location that can be passed to load_vectors.
    """
    # see wordvectors.ipynb for info on how these files were
    # generated from the original glove data.
    md5sums = {'6B.50d': '8e1557d1228decbda7db6dfd81cd9909',
               '6B.100d': 'c92dbbeacde2b0384a43014885a60b2c',
               '6B.200d': 'af271b46c04b0b2e41a84d8cd806178d',
               '6B.300d': '30290210376887dcc6d0a5a6374d8255'}
    glove_path = os.path.abspath('data/glove/results')
    %mkdir -p $glove_path
    return get_file(dataset,
                    'http://files.fast.ai/models/glove/' + dataset + '.tgz',
                    cache_subdir=glove_path,
                    md5_hash=md5sums.get(dataset, None),
                    untar=True)

In [27]:
def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

In [28]:
vecs, words, wordidx = load_vectors(get_glove_dataset('6B.50d'))

Untaring file...


In [34]:
words[0],vecs[0]

('the',
 array([  4.1800e-01,   2.4968e-01,  -4.1242e-01,   1.2170e-01,   3.4527e-01,  -4.4457e-02,
         -4.9688e-01,  -1.7862e-01,  -6.6023e-04,  -6.5660e-01,   2.7843e-01,  -1.4767e-01,
         -5.5677e-01,   1.4658e-01,  -9.5095e-03,   1.1658e-02,   1.0204e-01,  -1.2792e-01,
         -8.4430e-01,  -1.2181e-01,  -1.6801e-02,  -3.3279e-01,  -1.5520e-01,  -2.3131e-01,
         -1.9181e-01,  -1.8823e+00,  -7.6746e-01,   9.9051e-02,  -4.2125e-01,  -1.9526e-01,
          4.0071e+00,  -1.8594e-01,  -5.2287e-01,  -3.1681e-01,   5.9213e-04,   7.4449e-03,
          1.7778e-01,  -1.5897e-01,   1.2041e-02,  -5.4223e-02,  -2.9871e-01,  -1.5749e-01,
         -3.4758e-01,  -4.5637e-02,  -4.4251e-01,   1.8785e-01,   2.7849e-03,  -1.8411e-01,
         -1.1514e-01,  -7.8581e-01], dtype=float32))

In [39]:
words[200],vecs[200]

('according',
 array([  3.6750e-01,   1.7162e-01,   4.5661e-01,   2.2694e-01,   5.0477e-01,  -1.6938e-01,
         -7.2449e-01,  -6.0276e-01,   2.5607e-01,  -6.7345e-01,   3.2970e-01,  -2.8103e-01,
          1.5122e-01,  -6.4325e-01,   1.0454e+00,   2.8958e-03,  -5.1234e-01,  -3.3298e-01,
         -9.2862e-02,   2.4603e-01,   3.1475e-01,  -2.0641e-02,   5.5353e-01,  -1.9807e-01,
          1.1941e-01,  -1.3270e+00,  -6.5037e-01,  -4.6369e-01,  -8.6273e-01,   3.8967e-01,
          3.3200e+00,  -7.3484e-01,   1.0476e-01,  -6.2037e-01,  -2.5884e-01,  -3.9999e-01,
          1.4253e-01,  -1.1855e-01,   6.2405e-01,   7.0724e-01,  -1.1078e-01,   2.9246e-01,
          4.9381e-01,  -2.4960e-01,   2.0108e-03,  -4.1030e-01,  -6.2928e-01,   7.8374e-01,
          1.7455e-01,   1.7664e-01], dtype=float32))

In [40]:
words[1200],vecs[1200]

('dropped',
 array([-0.7344, -0.6601,  0.9814,  0.2593,  0.6737,  0.3727, -0.7779, -0.4258, -0.1626,  0.0722,
         0.4386, -0.5844, -0.5519, -0.2722,  0.4606,  0.129 , -1.1517, -0.557 , -1.4038, -0.4075,
         0.4225,  0.1336,  0.3566, -0.8419, -0.4795, -1.2175,  0.6958,  0.1539,  0.295 ,  0.1802,
         2.7856,  0.0378,  0.6695,  0.7081,  0.6018, -0.7407,  0.1792,  0.0196,  0.3351, -0.5519,
        -0.2797,  0.1091,  0.6023, -0.1357, -0.0467, -0.1145, -0.0183, -0.1155,  0.3803, -0.311 ], dtype=float32))

The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [41]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [42]:
emb = create_emb()

We pass our embedding matrix to the Embedding constructor, and set it to non-trainable.

In [43]:
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_len, dropout=0.2, 
              weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [25]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [27]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=12, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/12
25000/25000 [==============================] - 12s - loss: 0.4783 - acc: 0.7801 - val_loss: 0.4546 - val_acc: 0.8002
Epoch 2/12
25000/25000 [==============================] - 12s - loss: 0.4621 - acc: 0.7857 - val_loss: 0.4357 - val_acc: 0.8158
Epoch 3/12
25000/25000 [==============================] - 12s - loss: 0.4509 - acc: 0.7943 - val_loss: 0.4075 - val_acc: 0.8340
Epoch 4/12
25000/25000 [==============================] - 12s - loss: 0.4368 - acc: 0.8029 - val_loss: 0.4327 - val_acc: 0.8000
Epoch 5/12
25000/25000 [==============================] - 12s - loss: 0.4250 - acc: 0.8056 - val_loss: 0.4178 - val_acc: 0.8079
Epoch 6/12
25000/25000 [==============================] - 12s - loss: 0.4166 - acc: 0.8116 - val_loss: 0.3945 - val_acc: 0.8419
Epoch 7/12
25000/25000 [==============================] - 12s - loss: 0.4049 - acc: 0.8142 - val_loss: 0.3845 - val_acc: 0.8414
Epoch 8/12
25000/25000 [==============================

But let's fine-tune the embedding weights - especially since the words we couldn't find in glove just have random embeddings.

In [28]:
model.layers[0].trainable=True

In [29]:
model.optimizer.lr=1e-4

In [30]:
model.fit(trn, labels_train, validation_data=(test, labels_test), nb_epoch=1, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 12s - loss: 0.3713 - acc: 0.8326 - val_loss: 0.3967 - val_acc: 0.8192
